<h1>Coursera Capstone Project</h1>
<h1>The Battle of Neighborhoods (Week 4)</h1>

<h1><b>Data section</b></h1>

<h3>Question 2 - To Do</h3>
<p>Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare 
location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location 
data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is 
only Foursquare location data.</p>

In [3]:
# A description of the data and how it will be used to solve the problem. (15 marks)

<h2><b>2. Data Section</b></h2>
<h3><b>Data description and the sources used o solve the business problem.</b></h3>

<h2><b>Description of the Data</b></h2>
<h4>The following data is required to answer the issues of the problem:</h4>
<ul>
        <li>
            List of neighborhoods of Miami, Florida with their geodata (latitude and longitude).
        </li>
        <li>
           List of apartments for rent in Miami area with their addresses and price.
        </li>
        <li>
            List of Subway metro stations in Miami with their relative location.
        </li>
         <li>
            Preferably, a list of apartments for rent with additional information, such as price, address, area, # of beds
        </li>
        <li>
            Venues for each Miami neighborhoods ( that can be clustered).
        </li>
        <li>
            Venues for subway metro stations. 
        </li>
    </ul>

<h2><b>How data will help to solve the problem</b></h2>
<h4>The data will be used as follows:</h4>
<ul>
        <li>
        Use Foursquare and geopy data to map top 10 venues for all Miami neighborhoods and clustered in groups ( as per Course LAB).
    </li>
        <li>
        Use foursquare and geopy data to map the location of metro stations , separately and on top of the above clustered map in order 
        to be able to identify the venues and amenities near each metro station, or explore each public transport location separately
     </li>
        <li>
        Use Foursquare and geopy data to map the location of rental places, in some form, linked to the public transport locations.
     </li>
        <li>
        Create a map that depicts, for instance, the average rental price per square ft, around a radious of (1.5 km) around each subway station - or a
        similar metric. I will be able to quickly point to the popups to know the relative price per subway area.
     </li>
        <li>
        Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
     </li>
        <li>
        Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such
        as Metrorail, etc.
     </li>
    </ul>

<h4>This will help to evaluate the decision of renting and answer the below questions:</h4>
<ul>
        <li>
what is the cost of rent (per square ft) around a mile radius from each Metrorail station?
 </li>
        <li>
what is the area of Miami with best rental pricing that meets criteria established?
 </li>
        <li>
What are the venues of the two best places to live? How the prices compare?
 </li>
        <li>
How venues distribute among Miami neighborhoods and around metro stations?
 </li>
        <li>
Are there tradeoffs between size and price and location?
 </li>
        <li>
Any other interesting statistical data findings of the real estate and overall data.
     </li>
    </ul>

<h2>Reference of venues around current residence in Maryland for comparison to Miami.</h2>

In [8]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geocoder
!pip install geopy
import geocoder
import os
!pip install folium
import folium # map rendering library

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

print('Libraries imported.')

     |████████████████████████████████| 112kB 7.2MB/s eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    pa

<h4>Coordinate of Maryland residence.</h4>

In [10]:
# Owings Mills, Maryland 21112 United States of America
address = 'Owings Mills, Maryland'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Maryland residence are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Maryland residence are 39.4010753, -76.796299.


In [11]:
neighborhood_latitude= 39.4010753
neighborhood_longitude = -76.796299

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'LE31KB1HNQNMQCVJWAI50CEALQI4AN4G5MEV3MCZXK0FSLGD' # My Foursquare ID
CLIENT_SECRET = 'WI1QAVFE1N5RREOTPET0C4QQTI4ZMK41MDG4ZHE1BABJP4KW' # My Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LE31KB1HNQNMQCVJWAI50CEALQI4AN4G5MEV3MCZXK0FSLGD
CLIENT_SECRET: WI1QAVFE1N5RREOTPET0C4QQTI4ZMK41MDG4ZHE1BABJP4KW


In [15]:
url= 'https://api.foursquare.com/v2/venues/explore?&client_id=LE31KB1HNQNMQCVJWAI50CEALQI4AN4G5MEV3MCZXK0FSLGD&client_secret=WI1QAVFE1N5RREOTPET0C4QQTI4ZMK41MDG4ZHE1BABJP4KW&v=20180604&ll=39.4010753,-76.796299&radius=500&limit=100'
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LE31KB1HNQNMQCVJWAI50CEALQI4AN4G5MEV3MCZXK0FSLGD&client_secret=WI1QAVFE1N5RREOTPET0C4QQTI4ZMK41MDG4ZHE1BABJP4KW&v=20180604&ll=39.4010753,-76.796299&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
#results

<h4>Function that extracts the category of the venues near Owings Mills from the Foursquare lab.</h4>

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
SBnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SBnearby_venues =SBnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SBnearby_venues['venue.categories'] = SBnearby_venues.apply(get_category_type, axis=1)

# clean columns
SBnearby_venues.columns = [col.split(".")[-1] for col in SBnearby_venues.columns]

SBnearby_venues.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Taj Palace Owings MIlls,North Indian Restaurant,39.399413,-76.800307
1,Artful Gourmet Bistro,New American Restaurant,39.399300,-76.801364
2,Asian Kitchen,Asian Restaurant,39.399578,-76.799104
3,Red Robin Gourmet Burgers and Brews,Burger Joint,39.403242,-76.794221
4,Yuki Sushi,Sushi Restaurant,39.399449,-76.799871
5,Olive Garden,Italian Restaurant,39.404531,-76.793403
6,"Brookside's Pizzeria, The Original NY Pizza Co.",Pizza Place,39.399343,-76.800756
7,Baskin-Robbins,Ice Cream Shop,39.399234,-76.799813
8,APlus at Sunoco,Coffee Shop,39.400044,-76.798403
9,New York & Company,Women's Store,39.399051,-76.793292


<h4>Map of Owings Mills in Maryland with venues near residence place - for project reference</h4>

In [21]:

# create map of Southbank Melbourne using latitude and longitude values
map_sb = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SBnearby_venues['lat'], SBnearby_venues['lng'], SBnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sb)  
    
map_sb